In [3]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )
import numpy as np
import os
print("Check!")

Check!


In [4]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")

table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]
# get table-top frame in the world, here the dummy is for getting familiar with Drake
table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

# move the table for a bit
X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, -0.4, 0.0]
) # or similar: X_W_table = RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.])

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table) # 2nd arg relative to 1st arg transformed by X_W_table 
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))

plant.Finalize()
plant.set_name("table_with_box")

# now build the diagram
diagram = builder.Build()

In [5]:
context = diagram.CreateDefaultContext()
mutable_context = plant.GetMyContextFromRoot(context)

In [32]:
def ComputeContactTangentialVelocity(ad_plant, mutable_context, bodyA, bodyB, ptB, nhat_BA_W):
    frameA = ad_plant.GetFrameByName(bodyA)
    frameB = ad_plant.GetFrameByName(bodyB)
    V_AB_W = frameB.CalcSpatialVelocity(mutable_context, \
                                                frameA, \
                                                ad_plant.world_frame())
    print(f"V_AB_W shape is {np.array(V_AB_W)}")
    R_w_B = ad_plant.CalcRelativeTransform(mutable_context, \
                                                ad_plant.world_frame(), \
                                                frameB).rotation() # DCM from body B pose in {W}
    
    print(f"R_w_B shape is {np.array(R_w_B)}")
    p_BCb_W = R_w_B @ ptB # the contact points in {B} expressed in {W}
    print(f"p_BCb_w shape is {np.array(p_BCb_W)}")
    v_ACb_W = V_AB_W.Shift(p_BCb_W).translational() # compute the contact point's velocity in {A}
    print(f"v_ACb_W shape is {np.array(v_ACb_W)}")
    print(f"v_ACb_W type is {type(v_ACb_W)}")
    v_tang_ACb_W = v_ACb_W - np.dot(v_ACb_W, nhat_BA_W) * nhat_BA_W
    return v_tang_ACb_W

v = ComputeContactTangentialVelocity(plant, mutable_context, \
                                     "panda_link8", "base_link_cracker", \
                                     np.array([1.,2.,3.]), np.array([1.,2.,3.]))

V_AB_W shape is SpatialVelocity(
  w=[0.0, 0.0, 0.0],
  v=[0.0, 0.0, 0.0],
)
R_w_B shape is RotationMatrix([
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 0.0],
  [0.0, 0.0, 1.0],
])
p_BCb_w shape is [1. 2. 3.]
v_ACb_W shape is [0. 0. 0.]
v_ACb_W type is <class 'numpy.ndarray'>


In [31]:
a = np.array([1,2,3])
b = np.array([[1],[1],[1]])
np.dot(a, b)

array([6])

In [33]:
v

array([0., 0., 0.])

In [34]:
c = [[[1,2,3],[4,5,6]],[[7,8,9],[10,11,12]],[[13,14,15],[16,17,18]]]

In [35]:
c

[[[1, 2, 3], [4, 5, 6]],
 [[7, 8, 9], [10, 11, 12]],
 [[13, 14, 15], [16, 17, 18]]]

In [36]:
c = np.array(c)

In [38]:
c.shape

(3, 2, 3)

In [39]:
c

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])

In [40]:
vars = 3

In [41]:
vars

3

In [43]:
prog = MathematicalProgram()
qvars = prog.NewContinuousVariables(5, 2, "q")

In [44]:
qvars

array([[Variable('q(0,0)', Continuous), Variable('q(0,1)', Continuous)],
       [Variable('q(1,0)', Continuous), Variable('q(1,1)', Continuous)],
       [Variable('q(2,0)', Continuous), Variable('q(2,1)', Continuous)],
       [Variable('q(3,0)', Continuous), Variable('q(3,1)', Continuous)],
       [Variable('q(4,0)', Continuous), Variable('q(4,1)', Continuous)]],
      dtype=object)

In [45]:
type(qvars)

numpy.ndarray

In [46]:
vvars = prog.NewContinuousVariables(5, 2, "v")

In [47]:
a = np.array([1,2,3])
b = np.array([1,2,3])

In [48]:
a*b

array([1, 4, 9])

In [49]:
c

array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])

In [52]:
d = c.reshape((c.shape[0]*c.shape[1], c.shape[2]))

In [53]:
d

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

In [54]:
np.append([], [1,2,3])

array([1., 2., 3.])

In [56]:
np.append([1,2,3], [1,2,3], axis=0)

array([1, 2, 3, 1, 2, 3])